In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne 
from nice.markers import (KolmogorovComplexity, TimeLockedContrast, PowerSpectralDensityEstimator, 
                          PowerSpectralDensity, SymbolicMutualInformation, PermutationEntropy)

import pycsd

In [40]:
def all_markers(epochs, tmin, tmax):
        """
        Computes all ther markers for given epochs.
        epochs: the epochs from which to compute the markers
        
        """
        
        

        # =============================================================================
        # SPECTRAL MARKERS
        # =============================================================================
          #PowerSpectralDensityL
        psds_params = dict(n_fft=4096, n_overlap=100, n_jobs='auto', nperseg=128)
        base_psd = PowerSpectralDensityEstimator(
            psd_method='welch', tmin=tmin, tmax=tmax, fmin=1., fmax=45.,
            psd_params=psds_params, comment='default')


            #alpha normalized
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alphan')
        alpha.fit(epochs)
        from scipy.stats import trim_mean
        def trim_mean80(a, axis=0):
            return trim_mean(a, proportiontocut=.1, axis=axis)


        reduction_func = [{'axis': 'frequency', 'function': np.sum},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]
        #alpha normalized
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alpha')
        alpha.fit(epochs)
        dataalpha_n = alpha._reduce_to(reduction_func, target='epochs', picks=None)

        #alpha
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=False, comment='alpha')
        alpha.fit(epochs)
        dataalpha = alpha._reduce_to(reduction_func, target='epochs', picks=None)

        #delta normalized
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4.,normalize=True, comment='delta')
        delta.fit(epochs)
        datadelta_n = delta._reduce_to(reduction_func, target='epochs', picks=None)


        #delta
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4,normalize=False, comment='delta')
        delta.fit(epochs)
        datadelta = delta._reduce_to(reduction_func, target='epochs', picks=None)

        #theta normalized
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,normalize=True, comment='theta')
        theta.fit(epochs)
        datatheta_n = theta._reduce_to(reduction_func, target='epochs', picks=None)


        #theta
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8,normalize=False, comment='theta')
        theta.fit(epochs)
        datatheta = theta._reduce_to(reduction_func, target='epochs', picks=None)

        #gamma normalized
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45.,normalize=True, comment='gamma')
        gamma.fit(epochs)
        datagamma_n = gamma._reduce_to(reduction_func, target='epochs', picks=None)


        #gamma
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45,normalize=False, comment='theta')
        gamma.fit(epochs)
        datagamma = gamma._reduce_to(reduction_func, target='epochs', picks=None)

        #beta normalized
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30.,normalize=True, comment='beta')
        beta.fit(epochs)
        databetaa_n = beta._reduce_to(reduction_func, target='epochs', picks=None)


        #beta
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30,normalize=False, comment='beta')
        beta.fit(epochs)
        databeta = beta._reduce_to(reduction_func, target='epochs', picks=None)


        # =============================================================================
        # INFORMATION THEORY MARKERS
        # =============================================================================

        komplexity = KolmogorovComplexity(tmin=tmin, tmax=tmax, backend='openmp')
        komplexity.fit(epochs)
        komplexityobject=komplexity.data_ ###Object to save, number of channels*number of epochs, it's ndarray
        reduction_func= [{'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]


        datakomplexity = komplexity._reduce_to(reduction_func, target='epochs', picks=None)


        p_e = PermutationEntropy(tmin=tmin, tmax=tmax)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e = p_e._reduce_to(reduction_func, target='epochs', picks=None)

        # =============================================================================
        # wSMI MARKERS
        # =============================================================================
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=8, backend="openmp",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_

        reduction_func= [{'axis': 'channels_y', 'function': np.median},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]
        
        datawSMI = wSMI._reduce_to(reduction_func, target='epochs', picks={'epochs': None, 'channels': np.arange(64)})
        
        
        return {'wSMI':datawSMI, 'p_e':datap_e, 'k':datakomplexity, 'b':databeta,'b_n':databetaa_n, 'g':datagamma, 'g_n':datagamma_n, 't':datatheta,'t_n': datatheta_n , 'd':datadelta,
        'd_n':datadelta_n, 'a_n':dataalpha_n, 'a':dataalpha}


In [4]:
epoch_type = 'evoked'
# epoch_type = 'pseudo-rs'


all_participants = ['VP07','VP08','VP09', 'VP10','VP11','VP12','VP13','VP14','VP18','VP19','VP20','VP22','VP23','VP24','VP25','VP26','VP27','VP28','VP29','VP30','VP31','VP32','VP33','VP35','VP36','VP37']


path = '/media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/' #icm-linux
# path = '/Users/nicobruno/ownCloud/MW_eeg_data/minmarker/' #nico-mac

In [39]:
participant = all_participants[1]

folder = path + participant +'/'
epochs_subtracted = mne.read_epochs(folder + participant + epoch_type + 'ar-subtracted_epo.fif')

epochs_subtracted.info['description'] = 'biosemi/64'

epochs =  epochs_subtracted.pick_types(eeg = True)

SC = all_markers(epochs_subtracted['SC/go/correct'], 0, 0.6)

Reading /media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/VP08/VP08evokedar-subtracted_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
Autodetected number of jobs 24
Effective window size : 16.384 (s)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.7s remaining:    2.0s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.7s remaining:    0.9s
[Parallel(n_jobs=24)]: Done  16 out of  24 | elapsed:    0.8s remaining:    0.4s
[Parallel(n_jobs=24)]: Done  21 out of  24 | elapsed:    0.9s remaining:    0.1s


Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']


[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    0.9s finished


Running KolmogorovComplexity
Elapsed time 0.3233320713043213 sec
Reduction order for nice/marker/KolmogorovComplexity/default: ['channels']
Filtering  at 10.42 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Autodetected number of jobs 24
Computing CSD
Using 10-5 locations for CSD
Using 24 jobs


/home/nicolas.bruno/anaconda3/lib/python3.7/site-packages/nice-0.1.dev0-py3.7.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: divide by zero encountered in log
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
/home/nicolas.bruno/anaconda3/lib/python3.7/site-packages/nice-0.1.dev0-py3.7.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: invalid value encountered in multiply
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.


Filtering  at 10.42 Hz


[Parallel(n_jobs=24)]: Done  72 tasks      | elapsed:    0.2s
[Parallel(n_jobs=24)]: Done 224 out of 224 | elapsed:    0.3s finished


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']


In [41]:
for i in all_participants[:]:
    participant = i

    folder = path + participant +'/'
    epochs = mne.read_epochs(folder + participant + epoch_type + 'ar-subtracted_epo.fif')
    
    epochs.info['description'] = 'biosemi/64' #necessary for wSMI
    
    epochs =  epochs_subtracted.pick_types(eeg = True) #EOGs break everything
    
    try:
        np.save(folder +  participant + '_' + epoch_type + '_' + 'SC' ,all_markers(epochs['SC/go/correct'], 0, 0.6))
    except:
        pass

    try:
        np.save(folder +  participant + '_' + epoch_type + '_' + 'PC' ,all_markers(epochs['PC/go/correct'], 0, 0.6))
    except:
        pass


Reading /media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/VP07/VP07evokedar-subtracted_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
123 matching events found
No baseline correction applied
0 projection items activated
Autodetected number of jobs 24
Effective window size : 16.384 (s)


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    4.1s remaining:   12.3s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    4.2s remaining:    5.0s
[Parallel(n_jobs=24)]: Done  16 out of  24 | elapsed:    4.3s remaining:    2.2s
[Parallel(n_jobs=24)]: Done  21 out of  24 | elapsed:    4.4s remaining:    0.6s


Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Running KolmogorovComplexity


[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    4.6s finished


Elapsed time 0.32912755012512207 sec
Reduction order for nice/marker/KolmogorovComplexity/default: ['channels']
Filtering  at 10.42 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Autodetected number of jobs 24
Computing CSD
Using 10-5 locations for CSD
Using 24 jobs


...

Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Autodetected number of jobs 24
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['freque

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=24)]: Done  16 out of  24 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=24)]: Done  21 out of  24 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    0.2s finished


Reduction order for nice/marker/KolmogorovComplexity/default: ['channels']
Filtering  at 10.42 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Autodetected number of jobs 24
Computing CSD
Using 10-5 locations for CSD
Using 24 jobs
Filtering  at 10.42 Hz


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done  12 out of  29 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=24)]: Done  18 out of  29 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=24)]: Done  24 out of  29 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=24)]: Done  29 out of  29 | elapsed:    0.1s finished


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
